### PeMS数据为美国加州政府的公开数据，单个文件中包含的是一千余个监测站点的单日数据（24小时*每小时12条数据）

In [1]:
'''导入包'''
import pandas as pd
import numpy as np 
import csv 
import os 

## 1.单个txt文件解析测试

In [2]:
'''读入txt文件，并将长字符串拆分'''
txtfile = list()
with open('data_test/d03_text_station_5min_2020_01_01.txt','r') as f:
    res = f.readlines()
    for one_line in res:
        # 可以使用[:-1]来去除行末的换行符，并按逗号拆分
        txtfile.append(one_line[:-1].split(','))
txtfile = np.array(txtfile)
print(txtfile.shape)

(349344, 52)


In [3]:
'''将数据导入pandas的DataFrame'''
df = pd.DataFrame(data={'time' : txtfile[:, 0],  
                        'station' :  txtfile[:, 1],
                        'Total_Flow' : txtfile[:, 9]})
# 查看数据
df.head()

,time,station,Total_Flow
0,01/01/2020 00:00:00,308511,3
1,01/01/2020 00:00:00,308512,37
2,01/01/2020 00:00:00,311831,0
3,01/01/2020 00:00:00,311832,10
4,01/01/2020 00:00:00,311844,3


In [4]:
'''查看数据统计信息'''
df.describe()

,time,station,Total_Flow
count,349344,349344,349344
unique,288,1213,632
top,01/01/2020 21:00:00,314060,
freq,1213,288,34414


In [5]:
'''将数据表按站点、时间进行排序'''
# 所得内容为：第一个站点0-24点车流信息、第二个站点0-24点车流信息。。。以此类推，共计1213个站点。
sort_df = df.sort_values(by = ['station', 'time'])
sort_df.head(10)

,time,station,Total_Flow
1212,01/01/2020 00:00:00,3013051,0
2425,01/01/2020 00:05:00,3013051,2
3638,01/01/2020 00:10:00,3013051,13
4851,01/01/2020 00:15:00,3013051,9
6064,01/01/2020 00:20:00,3013051,11
7277,01/01/2020 00:25:00,3013051,11
8490,01/01/2020 00:30:00,3013051,19
9703,01/01/2020 00:35:00,3013051,11
10916,01/01/2020 00:40:00,3013051,10
12129,01/01/2020 00:45:00,3013051,24


In [6]:
'''保留唯一有用的车流量列，将空值替换为0，并存为整形'''
df_array=sort_df.values
data = list()
for i in df_array:
    Total_Flow = int(i[2]) if len(i[2]) != 0 else 0
    data.append(Total_Flow)
data = np.array(data)
print(data.shape)

(349344,)


## 2.批量数据处理

单日数据处理的中间结果中，已经形成了以站点为第一排序对象，以时间为第二排序对象的数据表，多日数据处理应在此基础上，按天的顺序将各个站点的数据进行拼接。

In [7]:
'''将txt中的数据读取出来存到数组里'''
def txt2array(filedir,station):
    txtfile = list()
    with open(filedir, 'r') as f:
        res = f.readlines()
        for one_line in res:
            # 可以使用[:-1]来去除行末的换行符，并按逗号拆分
            txtfile.append(one_line[:-1].split(','))
    txtfile = np.array(txtfile)
    return txtfile

In [8]:
'''对数组中原始数据进行筛选、排序，以列表形式返回'''
# 原始数据中既有‘’的空字符串，又有Nan的空值，都需要替换为0
def array2list(array, station):
    # 转入pandas中方便进行排序、筛选操作
    df = pd.DataFrame(data={'time' : array[:, 0],  'station' :  array[:, 1],  'Total_Flow' : array[:, 9]})
    
    # 查找指定站点的数据
    this_station = df[df['station'].isin([str(station)])]
    # 将Nan填充为0
    this_station = this_station.fillna(value=0)
    # 因为原始数据总体是按时间排序的，所以这里其实不用再排一次了
    # new_df = this_station.sort_values(by = 'time')
    
    this_station_array = this_station.values
    
    # 将‘’填充为0
    data = list()
    for i in this_station_array:
        time = i[0]
        station = i[1]
        dasum = i[2] if len(i[2]) != 0 else 0
        data.append([time, station, dasum])
    return data

In [9]:
'''main函数，输入为数据文件夹路径及所要的站点'''
def main(data_dir,station):
    Station = list()
    # 找到txt文件路径
    data_files_dir_list = os.listdir(data_dir)
    # 遍历txt文件
    for one_dir in data_files_dir_list:
        print('开始处理：', one_dir)
        filedir = os.path.join(data_dir, one_dir)
        
        # 实际处理部分
        txtfile_array = txt2array(filedir,station)
        this_station_list = array2list(txtfile_array,station)
        Station = Station + this_station_list
        
    Station = np.array(Station)
    print('共计：', Station.shape[0], '条数据')
    
    # 将处理好的数据保存至CSV文件
    save_data_dir = data_dir + '/' + str(station) + '.csv'
    with open(save_data_dir, 'w', newline='') as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow(['time', 'station', 'Total_Flow'])
        for i in Station:
            writer.writerow(i)
    print('ALL Done！')

In [10]:
data_dir = 'data_test'
main(data_dir, 3013051)

开始处理： d03_text_station_5min_2020_01_01.txt
开始处理： d03_text_station_5min_2020_01_02.txt
开始处理： d03_text_station_5min_2020_01_03.txt
开始处理： d03_text_station_5min_2020_01_04.txt
开始处理： d03_text_station_5min_2020_01_05.txt
开始处理： d03_text_station_5min_2020_01_06.txt
开始处理： d03_text_station_5min_2020_01_07.txt
开始处理： d03_text_station_5min_2020_01_08.txt
开始处理： d03_text_station_5min_2020_01_09.txt
开始处理： d03_text_station_5min_2020_01_10.txt
开始处理： d03_text_station_5min_2020_01_11.txt
开始处理： d03_text_station_5min_2020_01_12.txt
开始处理： d03_text_station_5min_2020_01_13.txt
开始处理： d03_text_station_5min_2020_01_14.txt
开始处理： d03_text_station_5min_2020_01_15.txt
开始处理： d03_text_station_5min_2020_01_16.txt
开始处理： d03_text_station_5min_2020_01_17.txt
开始处理： d03_text_station_5min_2020_01_18.txt
开始处理： d03_text_station_5min_2020_01_19.txt
开始处理： d03_text_station_5min_2020_01_20.txt
开始处理： d03_text_station_5min_2020_01_21.txt
开始处理： d03_text_station_5min_2020_01_22.txt
开始处理： d03_text_station_5min_2020_01_23.txt
开始处理： d03_t